In [116]:
# Import required libraries and dependencies
import pandas as pd
import hvplot.pandas
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

In [117]:
# Load the data into a Pandas DataFrame
df_market_data = pd.read_csv(
    "Resources/crypto_market_data.csv",
    index_col="coin_id")

# Display sample data
df_market_data.head()

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y
coin_id,,,,,,,
bitcoin,1.08388,7.60278,6.57509,7.67258,-3.25185,83.51840,37.51761
ethereum,0.22392,10.38134,4.80849,0.13169,-12.88890,186.77418,101.96023
tether,-0.21173,0.04935,0.00640,-0.04237,0.28037,-0.00542,0.01954
ripple,-0.37819,-0.60926,2.24984,0.23455,-17.55245,39.53888,-16.60193
bitcoin-cash,2.90585,17.09717,14.75334,15.74903,-13.71793,21.66042,14.49384


In [118]:
# Generate summary statistics
df_market_data.describe()

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y
count,41.000000,41.000000,41.000000,41.000000,41.000000,41.000000,41.000000
mean,-0.269686,4.497147,0.185787,1.545693,-0.094119,236.537432,347.667956
std,2.694793,6.375218,8.376939,26.344218,47.365803,435.225304,1247.842884
min,-13.527860,-6.094560,-18.158900,-34.705480,-44.822480,-0.392100,-17.567530
25%,-0.608970,0.047260,-5.026620,-10.438470,-25.907990,21.660420,0.406170
50%,-0.063410,3.296410,0.109740,-0.042370,-7.544550,83.905200,69.691950
75%,0.612090,7.602780,5.510740,4.578130,0.657260,216.177610,168.372510
max,4.840330,20.694590,24.239190,140.795700,223.064370,2227.927820,7852.089700


In [119]:
# Plot your data to see what's in your DataFrame
df_market_data.hvplot.line(
    width=800,
    height=400,
    rot=90
)

:NdOverlay   [Variable]
   :Curve   [coin_id]   (value)

---

### Prepare the Data

In [120]:
# Use the `StandardScaler()` module from scikit-learn to normalize the data from the CSV file
scaledData = StandardScaler().fit_transform(df_market_data)

In [121]:
# Create a DataFrame with the scaled data
scaledMarketDataDf = pd.DataFrame(
    scaledData,
    columns=df_market_data.columns
)

# Copy the crypto names from the original DataFrame
scaledMarketDataDf["coin_id"] = df_market_data.index

# Set the coin_id column as index
scaledMarketDataDf = scaledMarketDataDf.set_index("coin_id")

# Display the scaled DataFrame
scaledMarketDataDf.head()

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y
coin_id,,,,,,,
bitcoin,0.508529,0.493193,0.772200,0.235460,-0.067495,-0.355953,-0.251637
ethereum,0.185446,0.934445,0.558692,-0.054341,-0.273483,-0.115759,-0.199352
tether,0.021774,-0.706337,-0.021680,-0.061030,0.008005,-0.550247,-0.282061
ripple,-0.040764,-0.810928,0.249458,-0.050388,-0.373164,-0.458259,-0.295546
bitcoin-cash,1.193036,2.000959,1.760610,0.545842,-0.291203,-0.499848,-0.270317


---

### Find the Best Value for k Using the Original Scaled DataFrame.

In [122]:
# Create a list with the number of k-values from 1 to 11
kValues = list(range(1, 11))

In [123]:
# Create an empty list to store the inertia values
inertiaValues = []

# Create a for loop to compute the inertia with each possible value of k
# Inside the loop:
# 1. Create a KMeans model using the loop counter for the n_clusters
# 2. Fit the model to the data using `df_market_data_scaled`
# 3. Append the model.inertia_ to the inertia list
for k in kValues:
    kModel = KMeans(n_clusters=k, random_state=0, n_init='auto')
    kModel.fit(scaledMarketDataDf)
    inertiaValues.append(kModel.inertia_)
inertiaValues[:5]

[287.0,
 198.5718180979813,
 145.63574772192862,
 79.02243535120975,
 66.21778006000338]

In [124]:
# Create a dictionary with the data to plot the Elbow curve
elbowData = {
    "k": kValues,
    "inertia": inertiaValues
}

# Create a DataFrame with the data to plot the Elbow curve
elbowDF = pd.DataFrame(elbowData)
elbowDF.head()

,k,inertia
0,1,287.000000
1,2,198.571818
2,3,145.635748
3,4,79.022435
4,5,66.217780


In [72]:
# Plot a line chart with all the inertia values computed with
# the different values of k to visually identify the optimal value for k.
hvplot.extension("bokeh")
elbowPlot = elbowDF.hvplot.line(
    x='k',
    y='inertia',
    title = 'Elbow Curve',
    xticks=kValues
)
elbowPlot

:Curve   [k]   (inertia)

#### Answer the following question: 

**Question:** What is the best value for `k`?

**Answer:** teh best value for K is `4`

---

### Cluster Cryptocurrencies with K-means Using the Original Scaled DataFrame

In [73]:
# Initialize the K-Means model using the best value for k
kModel = KMeans(n_clusters=4, n_init='auto')

In [74]:
# Fit the K-Means model using the scaled DataFrame
kModel.fit(scaledMarketDataDf)

KMeans(n_clusters=4)

In [75]:
# Predict the clusters to group the cryptocurrencies using the scaled DataFrame
cryptoPredictions = kModel.predict(scaledMarketDataDf)

# Print the resulting array of cluster values.
cryptoPredictions

array([3, 3, 0, 0, 3, 3, 3, 3, 3, 0, 0, 0, 0, 3, 0, 3, 0, 0, 3, 0, 0, 3,
       0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 1, 3, 0, 0, 2, 0, 0, 0, 0],
      dtype=int32)

In [76]:
# Create a copy of the scaled DataFrame
scaledMarketDataPredictionsDF = scaledMarketDataDf.copy()

In [77]:
# Add a new column to the copy of the scaled DataFrame with the predicted clusters
scaledMarketDataPredictionsDF["cryptoCluster"] = cryptoPredictions

# Display the copy of the scaled DataFrame
scaledMarketDataPredictionsDF.head()

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y,cryptoCluster
coin_id,,,,,,,,
bitcoin,0.508529,0.493193,0.772200,0.235460,-0.067495,-0.355953,-0.251637,3
ethereum,0.185446,0.934445,0.558692,-0.054341,-0.273483,-0.115759,-0.199352,3
tether,0.021774,-0.706337,-0.021680,-0.061030,0.008005,-0.550247,-0.282061,0
ripple,-0.040764,-0.810928,0.249458,-0.050388,-0.373164,-0.458259,-0.295546,0
bitcoin-cash,1.193036,2.000959,1.760610,0.545842,-0.291203,-0.499848,-0.270317,3


In [78]:
# Create a scatter plot using hvPlot by setting
# `x="price_change_percentage_24h"` and `y="price_change_percentage_7d"`.
# Color the graph points with the labels found using K-Means and
# add the crypto name in the `hover_cols` parameter to identify
# the cryptocurrency represented by each data point.
hvplot.extension("bokeh")
clusterPlot = scaledMarketDataPredictionsDF.hvplot.scatter(
    x="price_change_percentage_24h",
    y="price_change_percentage_7d",
    by="cryptoCluster",
    hover_cols=["coin_id"],
    marker=["hex", "square", "cross", "triangle"],
    title="Crypto cluster"
)
clusterPlot

:NdOverlay   [cryptoCluster]
   :Scatter   [price_change_percentage_24h]   (price_change_percentage_7d,coin_id)

---

### Optimize Clusters with Principal Component Analysis.

In [79]:
# Create a PCA model instance and set `n_components=3`.
pcaModel = PCA(n_components = 3)

In [81]:
# Use the PCA model with `fit_transform` to reduce the original scaled DataFrame
# down to three principal components.
cryptoPCAData = pcaModel.fit_transform(scaledMarketDataDf)
# View the scaled PCA data
cryptoPCAData[:5]

array([[-0.60066733,  0.84276006,  0.46159457],
       [-0.45826071,  0.45846566,  0.95287678],
       [-0.43306981, -0.16812638, -0.64175193],
       [-0.47183495, -0.22266008, -0.47905316],
       [-1.15779997,  2.04120919,  1.85971527]])

In [82]:
# Retrieve the explained variance to determine how much information
# can be attributed to each principal component.
pcaModel.explained_variance_ratio_

array([0.3719856 , 0.34700813, 0.17603793])

In [83]:
# dump all features that contributed to the variance ratios of each PC
pd.DataFrame(
    pcaModel.components_,
    columns=scaledMarketDataDf.columns,
    index=['PCA1', 'PCA2', 'PCA3']
)

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y
PCA1,-0.416728,-0.102432,-0.009162,0.191523,0.320365,0.594468,0.568379
PCA2,0.358162,0.227432,0.540415,0.562182,0.434040,0.027735,-0.150789
PCA3,-0.218795,0.787670,0.349534,-0.181744,-0.361377,0.039733,0.210541


#### Answer the following question: 

**Question:** What is the total explained variance of the three principal components?

**Answer:** Total Explained Variance = `0.89503166`

In [85]:
# Create a new DataFrame with the PCA data.
pcaDF= pd.DataFrame(
    cryptoPCAData,
    columns = ['PCA1', 'PCA2', 'PCA3']
)

# Copy the crypto names from the original scaled DataFrame
pcaDF['coin_id'] = df_market_data.index

# Set the coin_id column as index
pcaDF = pcaDF.set_index('coin_id')

# Display the scaled PCA DataFrame
pcaDF.head()

,PCA1,PCA2,PCA3
coin_id,,,
bitcoin,-0.600667,0.842760,0.461595
ethereum,-0.458261,0.458466,0.952877
tether,-0.433070,-0.168126,-0.641752
ripple,-0.471835,-0.222660,-0.479053
bitcoin-cash,-1.157800,2.041209,1.859715


---

### Find the Best Value for k Using the Scaled PCA DataFrame

In [86]:
# Create a list with the number of k-values from 1 to 11
kValues = list(range(1, 11))

In [88]:
# Create an empty list to store the inertia values
inertiaValues = []

# Create a for loop to compute the inertia with each possible value of k
# Inside the loop:
# 1. Create a KMeans model using the loop counter for the n_clusters
# 2. Fit the model to the data using `df_market_data_pca`
# 3. Append the model.inertia_ to the inertia list
for k in kValues:
    kModel = KMeans(n_clusters=k, random_state=0, n_init='auto')
    kModel.fit(pcaDF)
    inertiaValues.append(kModel.inertia_)

inertiaValues[:5]

[256.8740855678926,
 168.81189630685233,
 115.91807663320071,
 49.66549665179738,
 41.35731380312857]

In [91]:
# Create a dictionary with the data to plot the Elbow curve
pcaElbow = {
    "k": kValues,
    "inertia": inertiaValues
}
# Create a DataFrame with the data to plot the Elbow curve
pcaElbowDF = pd.DataFrame(pcaElbow)
pcaElbowDF.head()

,k,inertia
0,1,256.874086
1,2,168.811896
2,3,115.918077
3,4,49.665497
4,5,41.357314


In [94]:
# Plot a line chart with all the inertia values computed with
# the different values of k to visually identify the optimal value for k.
hvplot.extension("bokeh")
pcaElbowPlot = pcaElbowDF.hvplot.line(
    x='k',
    y='inertia',
    title='elbow Curve Using PCA Data', 
    xticks=kValues
)
pcaElbowPlot

:Curve   [k]   (inertia)

#### Answer the following questions: 

* **Question:** What is the best value for `k` when using the PCA data?

  * **Answer:** the best value for `k` is `4`


* **Question:** Does it differ from the best k value found using the original data?

  * **Answer:** Both values are the same.

### Cluster Cryptocurrencies with K-means Using the Scaled PCA DataFrame

In [96]:
# Initialize the K-Means model using the best value for k
pcaKmodel = KMeans(n_clusters=4, n_init='auto')

In [98]:
# Fit the K-Means model using the PCA data
pcaKmodel.fit(pcaDF)

KMeans(n_clusters=4)

In [99]:
# Predict the clusters to group the cryptocurrencies using the scaled PCA DataFrame
pcaPredictions = pcaKmodel.predict(pcaDF)

# Print the resulting array of cluster values.
pcaPredictions

array([3, 3, 0, 0, 3, 3, 3, 3, 3, 0, 0, 0, 0, 3, 0, 3, 0, 0, 3, 0, 0, 3,
       0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 1, 3, 0, 0, 2, 0, 0, 0, 0],
      dtype=int32)

In [103]:
# Create a copy of the scaled PCA DataFrame
pcaPredictionsDF = pcaDF.copy()

# Add a new column to the copy of the PCA DataFrame with the predicted clusters
pcaPredictionsDF["PCA Predicted Cluster"] = pcaPredictions

# Display the copy of the scaled PCA DataFrame
pcaPredictionsDF.head()

,PCA1,PCA2,PCA3,PCA Predicted Cluster
coin_id,,,,
bitcoin,-0.600667,0.842760,0.461595,3
ethereum,-0.458261,0.458466,0.952877,3
tether,-0.433070,-0.168126,-0.641752,0
ripple,-0.471835,-0.222660,-0.479053,0
bitcoin-cash,-1.157800,2.041209,1.859715,3


In [111]:
# Create a scatter plot using hvPlot by setting
# `x="PC1"` and `y="PC2"`.
# Color the graph points with the labels found using K-Means and
# add the crypto name in the `hover_cols` parameter to identify
# the cryptocurrency represented by each data point.
hvplot.extension("bokeh")
clusterPCAPlot = pcaPredictionsDF.hvplot.scatter(
    x='PCA1', 
    y='PCA2',
    by='PCA Predicted Cluster',
    hover_cols=['coin_id'],
    marker=['hex', 'square', 'cross', 'triangle'],
    title="Crypto Clusters Using PCA"
)
clusterPCAPlot

:NdOverlay   [PCA Predicted Cluster]
   :Scatter   [PCA1]   (PCA2,coin_id)

### Visualize and Compare the Results

In this section, you will visually analyze the cluster analysis results by contrasting the outcome with and without using the optimization techniques.

In [114]:
# Composite plot to contrast the Elbow curves
elbowPlot + pcaElbowPlot

:Layout
   .Curve.I  :Curve   [k]   (inertia)
   .Curve.II :Curve   [k]   (inertia)

In [115]:
# Composite plot to contrast the clusters
clusterPlot + clusterPCAPlot

:Layout
   .NdOverlay.I  :NdOverlay   [cryptoCluster]
      :Scatter   [price_change_percentage_24h]   (price_change_percentage_7d,coin_id)
   .NdOverlay.II :NdOverlay   [PCA Predicted Cluster]
      :Scatter   [PCA1]   (PCA2,coin_id)

#### Answer the following question: 

  * **Question:** After visually analyzing the cluster analysis results, what is the impact of using fewer features to cluster the data using K-Means?

  * **Answer:** A better performace with fewer features can be achieved as long as the the principal components capture the mojority of the data's variance. PCA helps reduce teh complexity of the data while preserving the most important info which is often enough to maintain well defined cluster. 